# Modelado TIER 2: XGBoost

Este notebook se enfoca en entrenar y evaluar un modelo XGBoost.

## 1. Carga de Librerías y Datos

In [ ]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns

# Rutas de los datos preprocesados
X_train_path = '../data/processed/X_train.parquet'
y_train_path = '../data/processed/y_train.parquet'
X_val_path = '../data/processed/X_val.parquet'
y_val_path = '../data/processed/y_val.parquet'
X_test_path = '../data/processed/X_test.parquet'
feature_info_path = '../data/processed/feature_info.pkl'

# Cargar los datos
X_train = pd.read_parquet(X_train_path)
y_train = pd.read_parquet(y_train_path).squeeze()
X_val = pd.read_parquet(X_val_path)
y_val = pd.read_parquet(y_val_path).squeeze()
X_test = pd.read_parquet(X_test_path)

# Cargar información de las características
with open(feature_info_path, 'rb') as f:
    feature_info = pickle.load(f)

print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_val:", X_val.shape)
print("Forma de y_val:", y_val.shape)
print("Forma de X_test:", X_test.shape)
print("Características seleccionadas:", feature_info['selected_features'])

## 2. Definición y Entrenamiento del Modelo XGBoost

In [ ]:
# Crear DMatrix para XGBoost
# XGBoost puede manejar nombres de características con caracteres especiales si se configuran adecuadamente,
# pero es buena práctica asegurarse de que sean compatibles.
# Las características ya fueron preprocesadas, así que deberían estar bien.
dtrain = xgb.DMatrix(X_train[feature_info['selected_features']], label=y_train)
dval = xgb.DMatrix(X_val[feature_info['selected_features']], label=y_val)
dtest = xgb.DMatrix(X_test[feature_info['selected_features']])

# Parámetros de XGBoost (puedes ajustarlos o usar Optuna/GridSearch para optimizarlos)
# Calcular scale_pos_weight para manejar el desbalance de clases
scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc', # Métrica de evaluación
    'eta': 0.05, # learning_rate
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42,
    'scale_pos_weight': scale_pos_weight # Para clases desbalanceadas
}

print("Entrenando el modelo XGBoost...")
watchlist = [(dtrain, 'train'), (dval, 'eval')]
xgb_model = xgb.train(params,
                      dtrain,
                      num_boost_round=1000, # Número de iteraciones de boosting
                      evals=watchlist,
                      early_stopping_rounds=50, # Parar si la métrica de evaluación no mejora en 50 rondas
                      verbose_eval=100) # Imprimir evaluación cada 100 rondas
print("Entrenamiento completado.")

## 3. Evaluación del Modelo

In [ ]:
# Predicciones en el conjunto de validación
y_pred_proba_val = xgb_model.predict(dval, iteration_range=(0, xgb_model.best_iteration))
y_pred_val = (y_pred_proba_val > 0.5).astype(int) # Umbral de 0.5

# Calcular AUC-ROC
auc_roc = roc_auc_score(y_val, y_pred_proba_val)
print(f"AUC-ROC en el conjunto de validación: {auc_roc:.4f}")

# Mostrar reporte de clasificación
print("\nReporte de Clasificación en el conjunto de validación:")
print(classification_report(y_val, y_pred_val))

# Mostrar matriz de confusión
print("\nMatriz de Confusión en el conjunto de validación:")
cm = confusion_matrix(y_val, y_pred_val)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - XGBoost')
plt.show()

## 4. Importancia de las Características

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
xgb.plot_importance(xgb_model, ax=ax, max_num_features=20, importance_type='gain') # 'weight', 'gain', 'cover'
plt.title('Importancia de las Características - XGBoost (gain)')
plt.show()

# Obtener y mostrar como DataFrame
importance = xgb_model.get_score(importance_type='gain')
feature_importance_df = pd.DataFrame({'feature': list(importance.keys()), 'importance': list(importance.values())})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
print(feature_importance_df)

## 5. Guardar el Modelo (Opcional)

In [ ]:
# Guardar el modelo entrenado
model_path = '../models/xgboost_model.pkl'
# xgb_model.save_model(model_path.replace('.pkl', '.json')) # Formato nativo de XGBoost
with open(model_path, 'wb') as f:
    pickle.dump(xgb_model, f)
print(f"Modelo XGBoost guardado en: {model_path} (y como .json)")


# También podrías guardar las predicciones si es necesario para una submission
# y_pred_test_proba = xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration))
# submission_df = pd.read_csv('../data/sample_submission.csv')
# submission_df['Exited'] = y_pred_test_proba
# submission_df.to_csv('../submissions/submission_xgb.csv', index=False)
# print("Archivo de submission generado: ../submissions/submission_xgb.csv")

In [ ]:
# Guardar el modelo entrenado
model_path = '../models/xgboost_model.pkl'
# xgb_model.save_model(model_path.replace('.pkl', '.json')) # Formato nativo de XGBoost
with open(model_path, 'wb') as f:
    pickle.dump(xgb_model, f)
print(f"Modelo XGBoost guardado en: {model_path} (y como .json)")


# También podrías guardar las predicciones si es necesario para una submission
# y_pred_test_proba = xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration))
# submission_df = pd.read_csv('../data/sample_submission.csv')
# submission_df['Exited'] = y_pred_test_proba
# submission_df.to_csv('../submissions/submission_xgb.csv', index=False)
# print("Archivo de submission generado: ../submissions/submission_xgb.csv")